In [1]:
#from google.colab import drive
import numpy as np
import os
from os import listdir
from os.path import isfile, join, splitext
from PIL import Image, ImageOps
import numpy as np
import random
import math
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.regularizers import l1
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input
from matplotlib import pyplot
import h5py

In [ ]:
#If you didn't run the previous code that do the preprocessing -> run this part of code please
#DATASET USED FOR THE MODEL FOR TEMPO ESTIMATION
!gdown --id 1nzep5kiiLNjjoFgoMa-gvLSbIQIr18yP
!gdown --id 1GemCMmnmkiubcf77Q_uD4xuM90k7rGkx

path=['Image_spec_key.zip','Annotations_key.zip']
path_images=["Image_spec_key/GS","Image_spec_key/GTZAN"]
path_ann=["Annotations_key/GS","Annotations_key/GTZAN"]
from zipfile import ZipFile
for p in path:
    with ZipFile(p, 'r') as zipp:
        # printing all the contents of the zip file
        #zip.printdir()
        print('Extracting all the files now...')
        zipp.extractall()
        print('Done!')
flag=0

In [7]:
#If you have done the preprocessing using the previous notebook, please run this part of code
path_images=["Dataset/Key/GS/Image_spec","Dataset/Key/GTZAN/Image_spec"]
path_ann=["Dataset/Key/GS/Ann","Dataset/Key/GTZAN/Ann"]
flag=1

In [11]:
def split_train_test_validation(path_label,path_img):
    f_l=[]
    f_i=[]


    for i,l in zip(path_images,path_ann):
        f_l.extend([os.path.join(root, name) for root, dirs, files in os.walk(l) for name in files if name.endswith((".txt"))])
        f_i.extend([os.path.join(root, name) for root, dirs, files in os.walk(i) for name in files if name.endswith((".h5"))]) 

    if flag==0:

        f_l=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_l]
        f_i=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_i]
    if flag==1:
        f_l=[f.split('/')[2]+"/"+(f.split('/')[-1]).split('\\')[1] for f in f_l]
        f_i=[f.split('/')[2]+"/"+(f.split('/')[-1]).split('\\')[1] for f in f_i]
    f_l= np.asarray(f_l)
    f_i= np.asarray(f_i)
    print(f_l)
    print(f_i)
    print("Sorting both images and annotations...")
    for n in range(f_l.shape[0]):
        while True:
            
            s=splitext(f_l[n])[0]
            #print(s)
            if "lerch" in s:
                s=s[:len(s)-6]

            res=np.where(f_i==f"{str(s)}.h5")
            if(n==res[0]):
                break
            f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
    c=list(zip(f_l,f_i))
    random.shuffle(c)
    f_l,f_i=zip(*c)
    if flag==1:
        f_l=["Dataset/Key/"+f.split('/')[0]+'/Ann/'+f.split('/')[1] for f in f_l]
        f_i=["Dataset/Key/"+f.split('/')[0]+'/Image_spec/'+f.split('/')[1] for f in f_i]
    images=[]
    ann=[]
    print("Processing the images...")
    for f in f_i:
        h5f = h5py.File(f"{f}",'r')
        b = h5f['d'][:]
        images.append(np.asarray(b))
        print(b)
    print("Processing the annotations...")
    
    for an in f_l:
        
        with open(an) as f:
            ann.append(int(f.readlines()[0]))
    test_x=np.asarray(images[0:math.ceil(len(f_i)*0.2)])
    test_file=f_i[:math.ceil(len(f_i)*0.2)]
    test_y=np.array(ann[0:math.ceil(len(f_i)*0.2)])
    train_x=np.asarray(images[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
    train_y=np.array(ann[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
    val_x=np.asarray(images[len(test_x)+len(train_x):])
    val_y=np.array(ann[len(test_x)+len(train_x):])
    train_y = to_categorical(train_y, 24)
    test_y = to_categorical(test_y, 24) 
    val_y= to_categorical(val_y, 24)
    print(f"Split a total of {len(f_i)} samples into three sets with {len(train_x)} training, {len(val_x)} validation and {len(test_x)} test samples.")
    return test_x,test_y,train_x,train_y,val_x,val_y,test_file


x_test,y_test,x_train,y_train,x_val,y_val,test_file=split_train_test_validation(path_ann,path_images)


['GS/1004923.LOFI_1.txt' 'GS/1004923.LOFI_2.txt' 'GS/1004923.LOFI_3.txt'
 ... 'GTZAN/rock.00098_2.txt' 'GTZAN/rock.00099_1.txt'
 'GTZAN/rock.00099_2.txt']
['GS/1004923.LOFI_1.h5' 'GS/1004923.LOFI_2.h5' 'GS/1004923.LOFI_3.h5' ...
 'GTZAN/rock.00098_2.h5' 'GTZAN/rock.00099_1.h5' 'GTZAN/rock.00099_2.h5']
Sorting both images and annotations...
Processing the images...
[[ -5.176138   -4.879957   -5.2424636 ... -11.479137  -12.753672
  -10.961621 ]
 [ -8.109463   -8.427374   -6.3579183 ...  -8.216089   -9.823527
  -10.220304 ]
 [-12.864973  -10.593737   -6.488496  ...  -9.429592  -12.020707
  -10.736948 ]
 ...
 [-33.77332   -31.455286  -32.528305  ... -34.23962   -33.870296
  -28.77463  ]
 [-34.389027  -32.576633  -33.825012  ... -34.93459   -34.671894
  -30.117144 ]
 [-34.642467  -33.9299    -34.678146  ... -35.29331   -34.70935
  -30.177656 ]]
[[-17.121258 -16.440039 -14.753239 ... -17.32454  -18.981424 -16.727833]
 [-18.939417 -18.742847 -16.955751 ... -24.190441 -21.845135 -19.230797]
 [

  -23.71875  ]]
[[ -5.3746734  -6.202583   -1.2613469 ...  -5.8234224  -5.554491
   -6.2051764]
 [ -8.600156   -9.069833   -5.4629726 ... -11.087465  -11.140092
  -11.453935 ]
 [-10.273654  -11.142219   -8.665899  ... -12.44258   -14.188266
  -13.643481 ]
 ...
 [-41.528477  -46.072384  -45.041122  ... -31.600433  -30.901176
  -28.830427 ]
 [-42.08522   -47.39508   -47.15338   ... -30.853722  -30.042091
  -27.614614 ]
 [-43.987473  -48.459595  -48.955162  ... -30.911242  -29.462603
  -27.90516  ]]
[[ -7.0091987  -2.653435   -5.02229   ...  -7.209124   -8.104366
   -6.666429 ]
 [-11.156945   -9.281656  -12.539272  ... -15.629698  -13.405107
  -11.2184   ]
 [-13.028227  -12.943441  -13.884062  ... -13.570805  -14.322555
  -13.062285 ]
 ...
 [-27.6974    -27.163078  -28.857117  ... -34.40976   -23.970623
  -22.216492 ]
 [-27.677933  -27.106579  -28.266994  ... -28.464668  -27.03847
  -24.111492 ]
 [-27.544804  -26.015404  -28.095333  ... -27.994562  -24.810513
  -21.444818 ]]
[[ -3.3874166

  -30.188808 ]]
[[-11.945597  -11.396039   -5.829982  ...  -3.8324723  -3.5801857
   -2.8667529]
 [-10.737432   -6.87458    -5.8039937 ...  -8.697353  -10.415194
   -9.718782 ]
 [-12.446653   -9.226938   -9.269795  ...  -9.811539  -12.958098
  -11.210591 ]
 ...
 [-22.51791   -21.831522  -19.553495  ... -26.600172  -20.616323
  -20.867876 ]
 [-23.017164  -23.013447  -20.1639    ... -24.871752  -22.862104
  -21.296072 ]
 [-23.737713  -24.648281  -22.043097  ... -26.815304  -23.434235
  -22.923393 ]]
[[ -7.9858704  -8.343289   -9.321737  ...  -6.385743   -7.3133006
   -8.624746 ]
 [-10.059249  -11.428235  -12.407645  ...  -9.135429  -10.951381
  -14.148495 ]
 [ -7.8491545  -9.534181  -10.745072  ...  -7.1412153  -9.545596
  -10.647086 ]
 ...
 [-17.51408   -21.316507  -21.561928  ... -21.185032  -22.246353
  -25.203438 ]
 [-18.383116  -22.99698   -23.670372  ... -22.427914  -23.105282
  -25.440075 ]
 [-18.965607  -21.877592  -21.999973  ... -22.440298  -22.095922
  -25.753    ]]
[[ -3.8053

  -19.94024  ]]
[[ -6.558964   -4.400913   -2.5244591 ... -12.359794  -12.258616
  -13.617196 ]
 [-10.386953  -10.219849   -6.3242564 ... -11.694268  -12.159945
  -13.955245 ]
 [-14.41995   -11.959277  -10.984511  ... -11.766482  -12.936843
  -15.5119915]
 ...
 [-31.112247  -31.993141  -31.870333  ... -29.81052   -29.678331
  -25.889288 ]
 [-28.91341   -30.04416   -27.76308   ... -27.650705  -25.848492
  -23.092926 ]
 [-28.293749  -28.717237  -26.793213  ... -26.6378    -24.589588
  -22.942268 ]]
[[-13.076516  -10.976886   -8.937572  ...  -9.385192   -8.549987
   -4.0575495]
 [-12.751723  -12.760422  -10.754324  ... -10.3777     -7.4002934
   -3.6985443]
 [-16.05212   -13.461385  -13.806989  ... -12.259404   -7.7499137
   -6.659001 ]
 ...
 [-19.978146  -19.969826  -21.850187  ... -16.625328  -17.536335
  -19.67705  ]
 [-21.987436  -21.072294  -21.793951  ... -20.437849  -18.484951
  -18.843195 ]
 [-21.649048  -20.255093  -21.98039   ... -17.48771   -17.86422
  -19.402954 ]]
[[ -6.62459

  -31.342062 ]]
[[ -6.4323697  -7.246653   -6.8775177 ...  -4.887227   -5.5281773
   -8.636704 ]
 [-10.309477  -13.287751   -9.36033   ... -10.5602     -9.981065
  -10.392193 ]
 [-14.484995  -15.758235  -10.248607  ... -13.085575  -11.457204
  -11.635494 ]
 ...
 [-17.567524  -20.445826  -22.429802  ... -20.356853  -16.724968
  -19.319483 ]
 [-18.345695  -19.90957   -21.497257  ... -20.412867  -18.052374
  -20.67577  ]
 [-19.260992  -20.499998  -21.301943  ... -20.984142  -18.973509
  -21.708162 ]]
[[ -5.3498282  -6.693664   -6.5732107 ...  -9.710435   -8.795877
   -8.211999 ]
 [ -5.8573804  -4.7643533  -5.3306723 ...  -6.5691037  -6.8154655
   -7.9868855]
 [-10.713568   -9.517414  -10.24693   ... -12.276724  -10.101512
   -9.876937 ]
 ...
 [-23.041245  -24.08602   -21.872566  ... -24.399315  -22.17231
  -25.042833 ]
 [-22.101185  -24.849562  -22.373394  ... -23.808336  -22.603426
  -26.006609 ]
 [-20.461426  -24.372543  -22.202646  ... -25.561234  -22.637451
  -26.098267 ]]
[[-25.23841

...

[[ -5.5531054  -6.106361   -6.151925  ...  -6.5026693  -7.340389
   -8.011907 ]
 [ -8.3049     -8.502243   -8.1208315 ...  -7.335439   -8.002698
   -8.5078535]
 [ -7.0405393  -7.2610316  -7.8235617 ...  -6.6910543  -7.2240696
   -7.914121 ]
 ...
 [-36.362698  -39.17349   -41.017246  ... -41.399727  -41.115738
  -40.46409  ]
 [-40.690857  -43.75997   -45.002552  ... -45.62706   -44.802486
  -44.90676  ]
 [-43.434067  -44.237194  -45.25888   ... -45.857006  -46.272358
  -45.945854 ]]
[[ -4.6296606  -3.6257286  -4.333027  ... -19.073221  -17.60302
   -7.1331344]
 [ -4.8386316  -2.5701118  -4.26578   ... -19.179047  -17.05591
   -5.9534106]
 [ -7.209784   -6.531796   -6.670618  ... -17.970953  -16.56806
   -8.034194 ]
 ...
 [-31.277733  -24.594156  -26.022743  ... -33.68092   -29.900015
  -31.633625 ]
 [-32.209126  -24.025312  -26.336435  ... -33.269566  -30.26659
  -31.142962 ]
 [-33.314056  -24.146963  -27.450779  ... -32.719578  -31.058178
  -30.904833 ]]
[[ -7.956948   -7.5592923  -6.1

  -16.497774 ]]
[[ -4.8437614  -5.7927823  -5.673437  ...  -7.06069    -6.507154
   -8.546361 ]
 [ -7.3279514  -7.0794096  -5.6195116 ...  -7.335821   -8.000537
  -10.7595   ]
 [ -8.539259  -10.232144   -9.67313   ...  -6.929449   -8.281863
   -9.662867 ]
 ...
 [-16.150076  -17.774994  -18.77422   ... -20.375618  -22.246326
  -20.306215 ]
 [-18.439087  -20.19885   -19.500946  ... -21.833948  -23.470346
  -22.179756 ]
 [-21.21991   -21.357157  -20.498344  ... -21.855753  -23.020214
  -22.615864 ]]
[[ -3.5310078  -5.390276   -6.8864117 ... -14.316227   -3.9997234
   -1.8108662]
 [ -6.066458   -4.3955083  -6.677845  ... -12.855709   -3.2026746
   -6.8638053]
 [ -8.656607   -7.06905    -8.448089  ... -13.861765   -6.1183195
   -9.368796 ]
 ...
 [-26.71686   -20.975138  -20.169949  ... -20.47748   -21.43697
  -27.668644 ]
 [-25.687387  -18.783073  -19.223715  ... -21.00804   -21.530014
  -28.679243 ]
 [-25.590803  -19.735374  -19.768127  ... -21.65141   -22.379412
  -26.62985  ]]
[[ -6.9068

  -19.246887 ]]
[[-22.0982    -25.234303  -27.015446  ...  -2.3248794 -12.472714
   -8.576667 ]
 [-16.34231   -16.814142  -17.623295  ...  -6.313029  -14.517009
   -8.166065 ]
 [ -6.0945716  -6.459387   -8.400236  ...  -5.7964163  -8.824152
   -4.549993 ]
 ...
 [-22.034672  -22.174183  -23.17691   ... -36.746067  -37.966633
  -24.984142 ]
 [-24.900455  -23.94202   -26.894005  ... -38.13498   -39.539577
  -25.73243  ]
 [-26.54122   -24.915743  -29.342136  ... -39.95749   -40.445183
  -26.523668 ]]
[[-10.961429  -11.922399  -15.081183  ...  -5.0269732  -5.494936
   -8.998441 ]
 [ -7.5541096 -11.021244  -14.926199  ...  -3.9216318  -6.6341696
   -7.8632264]
 [-13.19681   -15.021396  -17.14014   ...  -4.6163135  -7.4200144
  -11.196601 ]
 ...
 [-32.525387  -27.813974  -28.467073  ... -20.806446  -25.347008
  -31.473984 ]
 [-32.914726  -28.769325  -30.469923  ... -22.063023  -27.84516
  -32.03513  ]
 [-35.73478   -34.91529   -32.097984  ... -23.000397  -29.284187
  -33.465893 ]]
[[ -5.40654

  -27.826508 ]]
[[-12.581656  -12.936716  -13.182985  ...  -6.355299   -7.853218
  -10.606937 ]
 [-17.9177    -20.2119    -18.256693  ...  -4.7300344  -6.54785
  -10.36855  ]
 [-20.004831  -20.682066  -18.258366  ... -10.427025  -16.264183
  -15.10755  ]
 ...
 [-28.3907    -29.667633  -31.940184  ... -27.88898   -28.246881
  -29.146936 ]
 [-28.625578  -31.567747  -31.673517  ... -28.151005  -29.199368
  -29.991003 ]
 [-26.351774  -27.53522   -28.911026  ... -27.501936  -27.318468
  -28.527004 ]]
[[ -8.474062   -8.869647   -8.745579  ...  -7.2566805  -7.56679
   -7.2377005]
 [ -7.8379025  -7.3744726  -3.2865138 ... -10.095081  -10.300999
   -9.377964 ]
 [-15.817     -14.368529   -5.618258  ... -13.256631  -14.147124
  -13.56307  ]
 ...
 [-29.306076  -25.21057   -18.15585   ... -18.86043   -19.806534
  -19.41225  ]
 [-27.31322   -25.767197  -19.442408  ... -19.750036  -19.543425
  -19.69094  ]
 [-27.548939  -24.952017  -17.420618  ... -18.795776  -18.907444
  -18.668009 ]]
[[ -1.4472833 

KeyboardInterrupt: 

In [17]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input


model = tf.keras.Sequential([
    layers.Conv2D(16, (5, 5), activation='relu', input_shape=(256, 40,1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (5, 5), activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)), 
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(16, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (5, 5), activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)),
    layers.Dense(24, activation='softmax')
])
model.summary()
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.055),loss='categorical_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train, epochs=75, batch_size=132,validation_data=(x_val, y_val),callbacks=[es,mc])


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_3. Consider increasing the input size. Received input shape [None, 23, 1, 16] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')

In [ ]:
#the accuracy is defined as the readme explain:
#Relative fifth: error of 50% (Ex. Label: C major , Prediction: G major)
#Relative minor/major: error of 75% (Ex. Label: C major, Prediction: A minor)
#Parallel minor/major: error of 90% (Ex. Label: C major, Prediciotn: C minor) 
def get_accuracy(model, true_x, true_y): 
    res = model.predict(true_x)
    res = np.argmax(res, axis=-1)
    acc = 0
    for i in range(len(true_y[:, 0])):
        r=res[i]
        t=np.where(true_y[i,:]==1)[0]
        if r==t:
          
          acc+=1
        elif r==(t+7)% (24//2):
          acc+=0.5
        elif r==(t-3) or r==(t+9)%24 or r==(t+3)%(24//2):
          acc+=0.25
        elif (r==(t+12)%24):
          acc+=0.1
        else:
            pass
    tot = len(true_y[:,0])
    print('True - total', acc, tot)
    print('acc: {}'.format((acc/tot)))

    
#This function is implemented in order to have the training dataset distinguished for the original dataset, indeed
#the output are the ExtendedBallroom and Giantsteps dataset
def get_dataset(label,img,test_file):
  f_l=[f for f in listdir("Annotations") if isfile(join("Annotations", f))]
  f_i=[f for f in listdir("Image_spec2") if isfile(join("Image_spec2", f))]
  f_l= np.asarray(f_l)
  f_i= np.asarray(f_i)
  for n in range(f_l.shape[0]):
    print(n)
    while True:
      s=splitext(f_l[n])[0]
      if "lerch" in s:
        s=s[:len(s)-6]
      res=np.where(f_i==f"{str(s)}.jpeg")
      if(n==res[0]):
        break
      f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
  c=list(zip(f_l,f_i))
  giant_i=[]
  gtzan_i=[]
  gtzan_l=[]
  giant_l=[]
  lofi="LOFI"
  words=['blues','classical','country','disco','hiphop','jazz','metal','rock','reggae','pop']
  for f in f_i:
    if f in test_file: 
      if lofi in f:
        idx=np.where(f_i == f)
        giant_i.append(f)
        giant_l.append(f_l[idx])
      for w in words:
        if w in f:
          idx=np.where(f_i == f)
          gtzan_i.append(f)
          gtzan_l.append(f_l[idx])
  gt_i=[]
  gt_l=[]
  gi_i=[]
  gi_l=[]
  for g1 in gtzan_i:
    gt_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{g1}").resize((100,100)))))
  for g2 in giant_i:  
    gi_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{g2}").resize((100,100)))))
  for g1 in gtzan_l:
    with open(f"Annotations/{g1[0]}") as f:
      gt_l.append(int(f.readlines()[0]))
  for g2 in giant_l:
    with open(f"Annotations/{g2[0]}") as f:
      gi_l.append(int(f.readlines()[0]))
  gt_i=np.asarray(gt_i)
  gt_l=np.asarray(gt_l)
  gi_i=np.asarray(gi_i)
  gi_l=np.asarray(gi_l)
  return gt_i,gt_l,gi_i,gi_l
gtzan_i,gtzan_l,giant_i,giant_l=get_dataset(path_ann,path_images,test_file)


In [ ]:
get_accuracy(model,giant_i,giant_l)
get_accuracy(model,gtzan_i,gtzan_l)